In [ ]:
from Raven.AoT import *
from Raven.BuildTree import *

In [ ]:
from DataAPI import *

In [ ]:
mc = MayaController()

In [ ]:
tree = build_single_person_scene_one_pose()

In [ ]:
#mc.Close()

In [ ]:
tree.children[0].children[0].children

In [ ]:
tree_pose = tree.children[0].children[0].children[2]

In [ ]:
tree.children

In [ ]:
pg = tree.sample()

In [ ]:
pg_pose = pg.children[0].children[0].children[2]

In [ ]:
pg_pose.children[0].head_leaf.selected_file

In [ ]:
mc.SetCurrentTimeFrame(0)

In [ ]:
mc.GenerateSceneFromPose(pg_pose.children[0].head_leaf.selected_file, "tempcc")

In [ ]:
mc.UndoToBeginning()

In [ ]:
pg.node_dict

In [ ]:
pg.node_dict['body'].head_leaf.selected_file

In [ ]:
rt = pg.node_dict['relative transform']

In [ ]:
position = rt.position_leaf.relativePosition

In [ ]:
mc.SetObjectLocalTransform("Main",[position[0], 0, position[1]])

In [ ]:
rotation = rt.rotation_leaf.relativeRotation

In [ ]:
mc.SetObjectLocalRotation("Main", [0, rotation, 0])

In [ ]:
scale = rt.scale_leaf.relativeScale

In [ ]:
scale

In [ ]:
mc.SetObjectAttribute("Main","scaleZ", scale)

------------------------------------------

In [1]:
from BuildAnimation import *

In [2]:
ab = AnimationBuilder(build_single_person_scene_two_poses)

In [3]:
ab.maya_controller.UndoToBeginning(max_step=1000)

(UndoToBeginning)Undo steps: 390


In [4]:
ab.BuildScene()

In [5]:
ab.BuildDefaultFrame()

In [6]:
endFrame_node = ab.pg.node_dict["end frame"]

In [7]:
ab.maya_controller.SetCurrentTimeFrame(endFrame_node.next_frame)

In [8]:
ab.BuildFrame(endFrame_node)

In [ ]:
ab.BuildTransform(transform_node)

In [ ]:
body_node = ab.pg.node_dict["init body"]

In [ ]:
emotion_node = ab.pg.node_dict["init emotion"]

In [ ]:
ab.BuildBody(body_node)

In [ ]:
ab.BuildEmotion(emotion_node)

In [ ]:
ab.SetKey()